In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import json
import base64
import requests
import numpy as np
# Inicializar el driver
driver = webdriver.Firefox()

URL = "https://eumostwanted.eu/"
driver.get(URL)
time.sleep(5)

# Cerrar el botón de cookies si existe
try:
    cookies = driver.find_element(By.ID, 'sort_close')
    cookies.click()
    time.sleep(2)
except Exception as e:
    print("No se encontró el botón de cookies:", e)

# Obtener los href de los elementos con clase 'wantedItem'
wanted_items = driver.find_elements(By.CLASS_NAME, 'wantedItem')
links = [item.get_attribute("href") for item in wanted_items if item.get_attribute("href")]

lista_datos = []

# Extraer datos de cada enlace
for href in links:
    driver.get(URL + href)
    
    try:
        driver_datos = driver.find_element(By.CLASS_NAME, "wanted_top_right")
        driver_imagen = driver.find_element(By.CLASS_NAME, "wanted_top_left")
        nombre_archivo = re.sub(r'\W+', '_', href) + ".png"

        driver.save_screenshot(nombre_archivo)

        
        # Funciones para extraer texto
        def extraer_texto_completo(clase):
            try:
                elemento = driver_datos.find_element(By.CLASS_NAME, clase)
                items = elemento.find_elements(By.CLASS_NAME, "field__item")
                return " ".join([item.text.strip() for item in items if item.text.strip()])
            except:
                return None

        def extraer_texto(clase):
            try:
                return driver_datos.find_element(By.CLASS_NAME, clase).text.strip()
            except:
                return None

        def extraer_imagen(clase):
            try:
                imagen_elemento = driver_imagen.find_element(By.CLASS_NAME, clase).find_element(By.TAG_NAME, "img")
                respuesta = requests.get(imagen_elemento.get_attribute("src"))
                return base64.b64encode(respuesta.content).decode("utf-8")
            except Exception as e:
                return  None

        # Extraer datos
        datos_persona = {
            "Nombre": extraer_texto("field--name-field-w-first-name"),
            "Alias": extraer_texto_completo("field--node-field-alias"),
            "Pais": extraer_texto("field--name-field-enfast-country"),
            "Crimen": extraer_texto_completo("field--name-field-crime"),
            "Sexo": extraer_texto_completo("field--name-field-gender"),
            "Altura": extraer_texto_completo("field--name-field-approximate-height"),
            "Color_ojos": extraer_texto_completo("field--name-field-eye-colour"),
            "Identificadores": extraer_texto_completo("field--node-field-characteristics"),
            "Fecha_nac": extraer_texto_completo("field--name-field-date-of-birth"),
            "Nacionalidad": extraer_texto_completo("field--name-field-nationality"),
            "Etnia": extraer_texto_completo("field--name-field-ethnic-origin"),
            "Idiomas": extraer_texto_completo("field--node-field-languages"),
            "Estado_caso": extraer_texto_completo("field--name-field-state-of-case"),
            "Fecha_publicacion": extraer_texto_completo("field--name-field-w-last-name"),
            "Imagen": extraer_imagen("field__item")
        }

        lista_datos.append(datos_persona)

    except Exception as e:
        print(f"Error al extraer datos de {href}:", e)


with open('masbuscados.json', 'w', encoding='utf-8') as f:
    json.dump(lista_datos, f, ensure_ascii=False, indent=4)

driver.close()




In [3]:
df = pd.read_json('masbuscados.json')

df.head()

,Nombre,Alias,Pais,Crimen,Sexo,Altura,Color_ojos,Identificadores,Fecha_nac,Nacionalidad,Etnia,Idiomas,Estado_caso,Fecha_publicacion,Imagen
0,"MUNTEANU, ADRIAN",None,by Romania,Participation in a criminal organisation Sexua...,Male,168 cm,Hazel,None,"August 5, 1983 (41 years)",Romanian,European,Romanian,"Sentenced to 11,8 years of prison","on November 16, 2023, last modified on Novembe...",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
1,"OMAIRAT, Walid",None,by Germany,"Murder, grievous bodily injury",Male,158 cm,Brown,None,"January 1, 1978 (47 years)",Lebanese,None,German Lebanese,Ongoing investigation,"on January 22, 2025, last modified on January ...",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
2,"RUSIŅŠ , Leons",None,by Latvia,"Murder, grievous bodily injury",Male,191 cm,None,None,"April 15, 1970 (54 years)",Latvian,None,English Latvian Russian,Ongoing investigation,"on April 29, 2023, last modified on May 25, 2023",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
3,"COTOS, ADRIAN",None,by Romania,Illicit trafficking in narcotic drugs and psyc...,Male,180 cm,Green,None,"September 2, 1983 (41 years)",Romanian,European,Romanian,"Sentenced to 10,5 years of prison","on November 16, 2023, last modified on Novembe...",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
4,"PANEK, DANIEL TOMASZ",None,by Slovenia,Rape,Male,176 - 180 cm,Blue,None,"August 14, 1976 (48 years)",Polish,European,Czech English Italian Polish,Failed to attend court,"on August 20, 2024, last modified on August 20...",iVBORw0KGgoAAAANSUhEUgAAAZAAAAH0CAYAAAAT2nuAAA...
